In [1]:
from fforma.metrics import WeightedPinballLoss
from fforma.meta_learner import MetaLearnerNN
from fforma.meta_results_r_data import prepare_fforma_data

## Meta results from R (hyndman, et al)

In [2]:
feats_train, X_models_train, y_models_train, feats_test, X_models_test = prepare_fforma_data('data/hyndman_data', dataset='Hourly') #dataset=None for all data

Successfully decompressed  data/hyndman_data/raw/decompressed_data


## QFFORMA

In [182]:
nn_params = {'layers': [200, 100, 50, 25, 10], 
             'dropout': 0.1, 
             'epochs': 300, 
             'batch_size': 2,
             'learning_rate': 0.001, 
             'gradient_eps': 1e-8, 
             'weight_decay': 0,
             'freq_of_test': 10,
             'loss_function': WeightedPinballLoss(0.4)}

In [183]:
model = MetaLearnerNN(nn_params, X_models_train.set_index(['unique_id', 'ds']), y_models_train, 48)

In [184]:
model.fit(feats_train.set_index(['unique_id']), verbose_eval=False)

Testing loss: 0.85529
Testing loss: 0.08053
Testing loss: 0.08908
Testing loss: 0.05553
Testing loss: 0.07591
Testing loss: 0.06328
Testing loss: 0.06205
Testing loss: 0.06503
Testing loss: 0.06597
Testing loss: 0.07121
Testing loss: 0.06303
Testing loss: 0.05687
Testing loss: 0.06100
Testing loss: 0.06145
Testing loss: 0.05401
Testing loss: 0.05458
Testing loss: 0.06023
Testing loss: 0.04922
Testing loss: 0.05901
Testing loss: 0.05460
Testing loss: 0.05589
Testing loss: 0.05755
Testing loss: 0.06608
Testing loss: 0.05629
Testing loss: 0.06077
Testing loss: 0.05947
Testing loss: 0.05983
Testing loss: 0.05564
Testing loss: 0.05828
Testing loss: 0.05358


In [185]:
feats_test_id = feats_test.set_index('unique_id')
X_models_test_id = X_models_test.set_index(['unique_id', 'ds'])

In [186]:
fforma_predictions = model.predict(feats_test_id)
fforma_predictions = pd.DataFrame(fforma_predictions, 
                                  index=feats_test_id.index, 
                                  columns=X_models_test_id.columns)

In [187]:
preds = (fforma_predictions * X_models_test_id).sum(1)
preds = preds.rename('y_hat').to_frame().reset_index()